In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
Berkeley = pd.read_csv('../../../../data/us/aggregate_berkeley.csv')                 
JHU = pd.read_csv('../../../../data/us/aggregate_jhu.csv')                 

In [6]:
def RatioToFrac(x):
    return x/(x+1)

def PercentToFrac(x):
    return x/100

def 

In [7]:
#Drop the first column which is meaningless
Berkeley = Berkeley.drop(columns = ['Unnamed: 0'])

In [8]:
#Improve readability
Berkeley = Berkeley.rename(columns = {'Population(Persons)2017': 'Population2017'})
Berkeley = Berkeley.rename(columns = {'PopTotalMale2017': 'PopMale2017'})
Berkeley = Berkeley.rename(columns = {'PopTotalFemale2017': 'PopFemale2017'})
Berkeley = Berkeley.rename(columns = {'MedianAge,Male2010': 'MedianAgeMale2010'})
Berkeley = Berkeley.rename(columns = {'MedianAge,Female2010': 'MedianAgeFemale2010'})
Berkeley = Berkeley.rename(columns = {'MedicareEnrollment,AgedTot2017':'MedicareEnrollmentTot2017'})
Berkeley = Berkeley.rename(columns = {'Smokers_Percentage':'SmokersPercentage'})
Berkeley = Berkeley.rename(columns = {'#FTEHospitalTotal2017':'#FTEHospital2017'})
Berkeley = Berkeley.rename(columns = {'TotalM.D.\'s,TotNon-FedandFed2017': '#MDs2017'})
Berkeley = Berkeley.rename(columns = {'#HospParticipatinginNetwork2017': '#HospitalsInNetwork2017'})
Berkeley = Berkeley.rename(columns = {'dem_to_rep_ratio':'FracDem'})

In [9]:
#Changing ratios and percentages to frac. Doing this whenever its relative to a population total.
#Editing to get dem_to_rep_ratio into a fraction of Democrats
Berkeley['FracDem'] = Berkeley['FracDem'].apply(RatioToFrac)

#Changing percentages to frac. 
Berkeley['DiabetesPercentage'] = Berkeley['DiabetesPercentage'].apply(PercentToFrac)
Berkeley['SmokersPercentage'] = Berkeley['SmokersPercentage'].apply(PercentToFrac)

In [10]:
Berkeley.to_csv('Aggregate_Berkeley.csv')

In [11]:
#Editing percentages to fractions. No ratios relative to a population total.
#Doing this whenever its a percentage out of a population total. Thus some percentages are still present, but they're relative to different quantities.

#Changing labels
JHU = JHU.rename(columns = {'Percent of adults with less than a high school diploma 2014-18': 'Frac of adults with less than a high school diploma 2014-18'})
JHU = JHU.rename(columns = {'Percent of adults with a high school diploma only 2014-18': 'Frac of adults with a high school diploma only 2014-18'})
JHU = JHU.rename(columns = {'Percent of adults completing some college or associate\'s degree 2014-18': 'Frac of adults completing some college or associate\'s degree 2014-18'}) 
JHU = JHU.rename(columns = {'Percent of adults with a bachelor\'s degree or higher 2014-18': 'Frac of adults with a bachelor\'s degree or higher 2014-18'})

#Changing the data
JHU['Frac of adults with less than a high school diploma 2014-18'] = JHU['Frac of adults with less than a high school diploma 2014-18'].apply(PercentToFrac)
JHU['Frac of adults with a high school diploma only 2014-18'] = JHU['Frac of adults with a high school diploma only 2014-18'].apply(PercentToFrac)
JHU['Frac of adults completing some college or associate\'s degree 2014-18'] = JHU['Frac of adults completing some college or associate\'s degree 2014-18'].apply(PercentToFrac)
JHU['Frac of adults with a bachelor\'s degree or higher 2014-18'] = JHU['Frac of adults with a bachelor\'s degree or higher 2014-18'].apply(PercentToFrac)

In [12]:

JHU[['Med_HH_Income_Percent_of_State_Total_2018']]

,Med_HH_Income_Percent_of_State_Total_2018
0,NaN
1,100.0
2,119.0
3,115.5
4,68.9
...,...
3268,NaN
3269,NaN
3270,NaN
3271,NaN


In [13]:
JHU.to_csv('Aggregate_JHU.csv')